In [39]:
import h5py
from skimage.feature import local_binary_pattern
import numpy as np
import torch
from torch import nn
import math, copy, time

In [40]:
f = h5py.File('dataset.h5')
f.keys()

<KeysViewHDF5 ['fitur_set', 'idx_cls', 'label_names', 'target_label']>

In [41]:
f.get('fitur_set')

<HDF5 dataset "fitur_set": shape (484, 224, 224, 3), type "<f2">

In [173]:
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])
    
class LBP(nn.Module):
    def __init__(self):
        super(LBP, self).__init__()
        self.radius = 1
        self.n_points = 8 * self.radius
        self.METHOD = 'default'
        
    def forward(self, data):
        _, w, h, d = data.shape
        lbp = lambda data: local_binary_pattern(data.reshape(w, h), self.n_points, self.radius, self.METHOD).reshape(w, h, 1)
        lbps = [[lbp(a[:,:,i]) for i in range(d)] for a in data]
        lbps = [np.concatenate(lbp, axis=2) / 255 for lbp in lbps]
        return torch.from_numpy(np.array(lbps)).float()

class Flatten2D(nn.Module):
    def __init__(self):
        super(Flatten2D, self).__init__()
        
    def forward(self, data):
        return torch.transpose(data.view(data.size(0), -1, data.size(3)), 1, 2)

class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, data):
        return data.view(data.size(0), -1)

class Argmax(nn.Module):
    def __init__(self):
        super(Argmax, self).__init__()
        
    def forward(self, data):
        return torch.argmax(data, dim=1)

class CNNLBP(nn.Module):
    def __init__(self):
        super(CNNLBP, self).__init__()
        cnn1d = torch.nn.Sequential(
            Flatten2D(),
            nn.Conv1d(3, 12, kernel_size=1, stride=1),
            nn.ReLU(),
            nn.Conv1d(12, 24, kernel_size=1, stride=1),
            nn.ReLU(),
            Flatten(),
            nn.Linear(24 * 1024, 128),
        )
        self.cnnrgb, self.cnnhsi = clones(cnn1d, 2)
        self.lbp = LBP()
        self.softmax = torch.nn.Sequential(
            Flatten(),
            nn.Linear(128 * 2, 6),
            nn.Softmax(dim=1),
        )
        
    def forward(self, data):
        rgb = data
        hsi = self.lbp(data)
        rgb = self.cnnrgb(rgb)
        hsi = self.cnnhsi(hsi)
        return self.softmax(torch.cat((rgb, hsi), dim=1))

In [182]:
x = torch.randn(10, 32, 32, 3) / math.e / 2 + 0.5
y = nn.functional.one_hot(torch.arange(0, 10) % 6).float()
model = CNNLBP()
y_pred = model(x)
y_pred, y

(tensor([[0.1663, 0.1694, 0.1784, 0.1588, 0.1703, 0.1567],
         [0.1648, 0.1699, 0.1783, 0.1580, 0.1721, 0.1570],
         [0.1654, 0.1709, 0.1787, 0.1570, 0.1691, 0.1590],
         [0.1640, 0.1691, 0.1768, 0.1601, 0.1711, 0.1588],
         [0.1621, 0.1690, 0.1808, 0.1551, 0.1734, 0.1596],
         [0.1651, 0.1726, 0.1750, 0.1580, 0.1711, 0.1582],
         [0.1646, 0.1694, 0.1776, 0.1582, 0.1718, 0.1586],
         [0.1631, 0.1710, 0.1826, 0.1561, 0.1713, 0.1558],
         [0.1641, 0.1695, 0.1783, 0.1576, 0.1693, 0.1613],
         [0.1681, 0.1685, 0.1770, 0.1572, 0.1730, 0.1562]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.]]))

In [185]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
running_loss, last_loss = 0., 0.
model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if epoch % 10 == 0:
        print('Epoch: {}, Loss: {}'.format(epoch, running_loss / 10))
        running_loss = 0.

Epoch: 0, Loss: 0.17544097900390626
Epoch: 10, Loss: 1.753701674938202
Epoch: 20, Loss: 1.7510527491569519
Epoch: 30, Loss: 1.7474077224731446
Epoch: 40, Loss: 1.743290936946869
Epoch: 50, Loss: 1.7388588190078735
Epoch: 60, Loss: 1.7341361045837402
Epoch: 70, Loss: 1.729101872444153
Epoch: 80, Loss: 1.7237156748771667
Epoch: 90, Loss: 1.7179197192192077
Epoch: 100, Loss: 1.7116468667984008
Epoch: 110, Loss: 1.7048240184783936
Epoch: 120, Loss: 1.697361207008362
Epoch: 130, Loss: 1.689153504371643
Epoch: 140, Loss: 1.6800748348236083
Epoch: 150, Loss: 1.6699762105941773
Epoch: 160, Loss: 1.6586970448493958
Epoch: 170, Loss: 1.646040952205658
Epoch: 180, Loss: 1.6317957043647766
Epoch: 190, Loss: 1.6157469630241394
Epoch: 200, Loss: 1.5976421594619752
Epoch: 210, Loss: 1.5772651433944702
Epoch: 220, Loss: 1.5544533133506775
Epoch: 230, Loss: 1.5291308641433716
Epoch: 240, Loss: 1.5014620304107666
Epoch: 250, Loss: 1.4719309091567994
Epoch: 260, Loss: 1.4414178490638734
Epoch: 270, Loss: